# ชุด Source code นี้ใช้ทดสอบ ผลลัพธ์ของ "python ingre.py"
# หรือการรัน script phython เพื่ออ่านข้อมูลจากไฟล์เอกสาร และ เก็บลงใน ChromaDb (AI DB)

In [71]:
from constants import (
    get_default_ai_db_collection_name,
    get_default_embedding,
    load_chroma_database,
    is_exists_collection_name,
    AI_DB_SEARCH_RESULT_RECORD,
    AI_DB_METADATA_DOCUMENT_SOURCE_NAME,
    AI_DB_METADATA_INTERNAL_IDX_NAME
)

db = load_chroma_database()
collection = db.get_collection(name=get_default_ai_db_collection_name(), embedding_function=get_default_embedding())

In [78]:
import chromadb

def query_from_funetune(query_str: str, db: chromadb.API) -> str:
    """
    ค้นหาเนื้อหาภายในเอกสาร ที่เก็บภายใน AI DB มีรูปประโยคใดบ้าง ที่ใกล้เคียงกับ query_str
    """
    ret_str = ''
    if is_exists_collection_name(collection_name=get_default_ai_db_collection_name(), db=db) == False:
        return ret_str
    
    collection = db.get_collection(
        name=get_default_ai_db_collection_name(), 
        embedding_function=get_default_embedding()
    )

    # Trim space ซ้าย/ขวา ก่อนใช้เป็นเงื่อนไขในการค้นหา เพราะทุกตัวอักษรถูกนำไปคิดเป็น เงื่อนไขทั้งหมด
    query_str = query_str.strip()

    # ค้นหารูปแบบประโยคที่เฉพาะเจาะจง
    parent_out = collection.query(
        query_texts=[query_str],
        n_results=AI_DB_SEARCH_RESULT_RECORD,
        where_document={
            '$contains': query_str
        }
    )
    print(parent_out)
    print(parent_out['metadatas'])

    if len(parent_out['documents'][0]) == 0:
        # ถ้าค้นหาด้วยเงื่อนไข รูปแบบประโยคที่เฉพาะเจาะจงไม่เจอ ให้ค้นหาแบบ distancing
        # คือ ให้ AI DB จัดลำดับข้อความที่มีความน่าจะเป็น ที่ใกล้เคียงกับ ประโยคคำถามมากที่สุด ไปหาน้อย (Distancing)
        out = collection.query(
            query_texts=[query_str],
            n_results=AI_DB_SEARCH_RESULT_RECORD
        )
        
        print(out)
        ret_str = '\n'.join([document_content for document_content in out['documents'][0]]) if len(out['documents'][0]) > 0 else ''
        return ret_str
    

    ret_str = parent_out['documents'][0][0]

    # ค้นหา Chunk ในลำดับถัดไปจาก parent_out
    # เนื่องจาก Chunk เป็นการตัดชุดของข้อความออกเป็นส่วนๆ มีโอกาสที่ คำตอบ ของคำถามจะไปอยู่ใช้ Chunk ลำดับถัดไป
    doc_source = parent_out['metadatas'][0][0][AI_DB_METADATA_DOCUMENT_SOURCE_NAME]
    doc_internal_idx = parent_out['metadatas'][0][0][AI_DB_METADATA_INTERNAL_IDX_NAME]
    doc_internal_idx1 = f'{doc_internal_idx[0:len(doc_internal_idx)-1]}{int(doc_internal_idx[-1]) + 1}'
    

    out = collection.query(
        query_texts=[query_str],
        n_results=1,
        where={
            '$and':[
                {
                    AI_DB_METADATA_DOCUMENT_SOURCE_NAME: {
                        '$eq': doc_source
                    }
                },
                {
                    AI_DB_METADATA_INTERNAL_IDX_NAME: {
                        '$eq': doc_internal_idx1
                    }
                }
            ]
        }
    )

    print(f'doc_source: {doc_source}, doc_internal_idx: {doc_internal_idx}, doc_internal_idx1: {doc_internal_idx1}')
    print(out)
    ret_str = f"{ret_str}\n{out['documents'][0][0]}" if len(out['documents'][0]) > 0 else ret_str
    return ret_str

In [81]:
query_str = "Can I deposit cash into someone else's account?"
print(AI_DB_SEARCH_RESULT_RECORD)
print(query_str)
ret = query_from_funetune(query_str=query_str, db=db)
print(ret)

2
Can I deposit cash into someone else's account?


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\chair\AppData\Local\Temp\ipykernel_30664\2380832635.py:4 in <module>                    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\chair\\AppData\\Local\\Temp\\ipykernel_30664\\2380832635.py'                         │
│                                                                                                  │
│ C:\Users\chair\AppData\Local\Temp\ipykernel_30664\87284537.py:20 in query_from_funetune          │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\chair\\AppData\\Local\\Temp\\ipykernel_30664\\87284537.py'                           │
│                                                                                                  │
│ d:\Program Files\Python310\lib\site-packages\chromadb\api\models\Collection.py:223 in query      │
│                                                                                                  │
│   220 │   │   if where_document is None:                                                         │
│   221 │   │   │   where_document = {}                                                            │
│   222 │   │                                                                                      │
│ ❱ 223 │   │   return self._client._query(                                                        │
│   224 │   │   │   collection_id=self.id,                                                         │
│   225 │   │   │   query_embeddings=query_embeddings,                                             │
│   226 │   │   │   n_results=n_results,                                                           │
│                                                                                                  │
│ d:\Program Files\Python310\lib\site-packages\chromadb\api\local.py:457 in _query                 │
│                                                                                                  │
│   454 │   │   where_document: WhereDocument = {},                                                │
│   455 │   │   include: Include = ["documents", "metadatas", "distances"],                        │
│   456 │   ) -> QueryResult:                                                                      │
│ ❱ 457 │   │   uuids, distances = self._db.get_nearest_neighbors(                                 │
│   458 │   │   │   collection_uuid=collection_id,                                                 │
│   459 │   │   │   where=where,                                                                   │
│   460 │   │   │   where_document=where_document,                                                 │
│                                                                                                  │
│ d:\Program Files\Python310\lib\site-packages\chromadb\db\clickhouse.py:596 in                    │
│ get_nearest_neighbors                                                                            │
│                                                                                                  │
│   593 │   │   │   raise TypeError("Argument collection_uuid cannot be None")                     │
│   594 │   │                                                                                      │
│   595 │   │   if len(where) != 0 or len(where_document) != 0:                                    │
│ ❱ 596 │   │   │   results = self.get(                                                            │
│   597 │   │   │   │   collection_uuid=collection_uuid,                                           │
│   598 │   │   │   │   where=where,                         

In [24]:
query_text = 'What does Lock(sell) mean?'
ret = collection.query(
    query_texts=[query_text],

    # อ้างอิงเอกสารจาก Link: https://docs.trychroma.com/usage-guide#using-where-filters
    where={
        '$or':[
            {
                'internal_idx':{
                    '$eq': 'idx_9'
                },
            },
             {
                'internal_idx':{
                    '$eq': 'idx_8'
                },
            }
        ]
    },
    # where_document={
    #     '$contains': query_text
    # }
)

print(ret)
print(ret['documents'][0])
print(ret['documents'][0][0])
print(ret['documents'][0][1])

{'ids': [['source_documents-single\\GS_QA.pdf-idx_9', 'source_documents-single\\GS_QA.pdf-idx_8']], 'embeddings': None, 'documents': [['Q22 Q: Are the Buy and Sell options both always available when the Account Status is\nActive?\nA22 A: No. It is not always the case, as it depends on the Share Status.\n\nQ24 Q: What does Lock(buy) mean?\nA24 A: Lock(buy) means that the customer is temporarily not allowed to buy securities even if the\nShare Status is Active.\n\nQ25 Q: What does Lock(sell) mean?\nA25 A: Lock(sell) means that the customer is temporarily not allowed to sell securities even if the\nShare Status is Active.', 'Q19 Q: What does HALTED Share Status mean?\nA19 A: HALTED Share Status means that buying and selling stocks have been suspended for a\ncertain security or underlying asset.\n\nQ20 Q: What does ACTIVE Share Status mean?\nA20 A: ACTIVE Share Status means that buying and selling stocks is allowed for a certain security\nor underlying asset.\n\nQ21 Q: What does PENDING Ac